In [ ]:
# Check GPU
!nvidia-smi

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Clone your repo (or upload files)
# Option 1: Clone from GitHub
!git clone https://github.com/Surya-Narayan-M/kortex_5th_sem.git
%cd kortex_5th_sem/sign_to_text

# Option 2: If files are in Drive, copy them
# !cp -r /content/drive/MyDrive/kortex_5th_sem .

In [ ]:
# Install dependencies
!pip install tqdm pandas numpy scipy

In [ ]:
# Setup paths and preprocess data in Colab
import os
import sys
from pathlib import Path
import numpy as np
from tqdm import tqdm
from scipy.ndimage import gaussian_filter1d

# Add project to path
sys.path.insert(0, '/content/kortex_5th_sem/sign_to_text')

# ========== PATHS ==========
DRIVE_RAW_DATA = "/content/drive/MyDrive/kortex_data/output"  # Raw landmarks (2.6 GB)
OUTPUT_PREPROCESSED = "/content/output_preprocessed"  # Where preprocessed data goes
CSV_PATH = "/content/drive/MyDrive/kortex_data/iSign_v1.1.csv"

# Create output directory
os.makedirs(OUTPUT_PREPROCESSED, exist_ok=True)

# Check if raw data exists
if not os.path.exists(DRIVE_RAW_DATA):
    print(f"⚠️ Raw data not found at {DRIVE_RAW_DATA}")
    print("Upload to: MyDrive/kortex_data/output/")
    print("(the 2.6 GB folder with .npy files from E:\\5thsem el\\output\\)")
else:
    print(f"✅ Found raw data at {DRIVE_RAW_DATA}")
    print(f"Files: {len(os.listdir(DRIVE_RAW_DATA))}")


In [ ]:
class ColabConfig:
    """Training configuration for Colab GPU (T4/V100)"""
    
    # Data paths - COLAB PATHS (HDF5 file instead of individual .npy files)
    h5_file = Path("/content/dataset.h5")  # ← NEW: HDF5 file path
    csv_path = Path("/content/iSign_v1.1.csv")
    vocab_path = Path("/content/kortex_5th_sem/sign_to_text/vocabulary.pkl")
    checkpoint_dir = Path("/content/drive/MyDrive/kortex_checkpoints")  # Save to Drive!

In [ ]:
# ========== PREPROCESSING FUNCTIONS (from preprocess.py) ==========

class PreprocessConfig:
    smoothing_sigma = 1.0
    min_hand_span = 0.01
    add_velocity = True
    add_acceleration = True

def normalize_landmarks(landmarks: np.ndarray) -> np.ndarray:
    """Normalize landmarks to be person and camera invariant"""
    T = landmarks.shape[0]
    
    # Reshape: hands (126) + shoulders (6) + elbows (6)
    hands = landmarks[:, :126].reshape(T, 2, 21, 3).copy()
    shoulders = landmarks[:, 126:132].reshape(T, 2, 3).copy()
    elbows = landmarks[:, 132:138].reshape(T, 2, 3).copy()
    
    # 1. Normalize each hand to wrist origin
    for h in range(2):
        wrist = hands[:, h, 0:1, :].copy()
        hands[:, h] = hands[:, h] - wrist
    
    # 2. Scale by hand span (wrist to middle fingertip)
    for h in range(2):
        span = np.linalg.norm(hands[:, h, 12] - hands[:, h, 0], axis=1, keepdims=True)
        span = np.maximum(span, PreprocessConfig.min_hand_span)
        hands[:, h] = hands[:, h] / span[:, :, np.newaxis]
    
    # 3. Normalize shoulders/elbows relative to body center
    body_center = shoulders.mean(axis=1, keepdims=True)
    shoulders = shoulders - body_center
    elbows = elbows - body_center
    
    # 4. Scale body landmarks by shoulder width
    shoulder_width = np.linalg.norm(shoulders[:, 0] - shoulders[:, 1], axis=1, keepdims=True)
    shoulder_width = np.maximum(shoulder_width, PreprocessConfig.min_hand_span)
    shoulders = shoulders / shoulder_width[:, :, np.newaxis]
    elbows = elbows / shoulder_width[:, :, np.newaxis]
    
    # 5. Flatten back
    normalized = np.concatenate([hands.reshape(T, -1), shoulders.reshape(T, -1), elbows.reshape(T, -1)], axis=1)
    return normalized

def compute_velocity(features: np.ndarray) -> np.ndarray:
    """Compute velocity (first-order derivative)"""
    velocity = np.zeros_like(features)
    velocity[1:] = features[1:] - features[:-1]
    return velocity

def compute_acceleration(velocity: np.ndarray) -> np.ndarray:
    """Compute acceleration (second-order derivative)"""
    acceleration = np.zeros_like(velocity)
    acceleration[1:] = velocity[1:] - velocity[:-1]
    return acceleration

def smooth_features(features: np.ndarray, sigma: float = 1.0) -> np.ndarray:
    """Apply Gaussian smoothing to reduce MediaPipe jitter"""
    if sigma <= 0 or features.shape[0] < 3:
        return features
    return gaussian_filter1d(features, sigma=sigma, axis=0, mode='nearest')

def preprocess_single_file(landmarks: np.ndarray, 
                           add_velocity: bool = True,
                           add_acceleration: bool = True,
                           smooth: bool = True) -> np.ndarray:
    """Complete preprocessing pipeline"""
    if landmarks.shape[0] == 0:
        return landmarks
    
    if landmarks.shape[0] < 3:
        pad_length = 3 - landmarks.shape[0]
        landmarks = np.pad(landmarks, ((0, pad_length), (0, 0)), mode='edge')
    
    # 1. Normalize
    normalized = normalize_landmarks(landmarks)
    
    # 2. Smooth
    if smooth:
        normalized = smooth_features(normalized, sigma=PreprocessConfig.smoothing_sigma)
    
    # 3. Velocity
    if add_velocity:
        velocity = compute_velocity(normalized)
        if smooth:
            velocity = smooth_features(velocity, sigma=PreprocessConfig.smoothing_sigma)
    
    # 4. Acceleration
    if add_acceleration and add_velocity:
        acceleration = compute_acceleration(velocity)
        if smooth:
            acceleration = smooth_features(acceleration, sigma=PreprocessConfig.smoothing_sigma)
    
    # 5. Concatenate
    features = [normalized]
    if add_velocity:
        features.append(velocity)
    if add_acceleration and add_velocity:
        features.append(acceleration)
    
    return np.concatenate(features, axis=1)

print("✅ Preprocessing functions loaded!")


In [ ]:
# ========== RUN PREPROCESSING ==========
import shutil

# Check if preprocessing already done
existing_files = len(os.listdir(OUTPUT_PREPROCESSED))
raw_files = len(os.listdir(DRIVE_RAW_DATA))

if existing_files == raw_files:
    print(f"✅ Preprocessing already complete! {existing_files} files found.")
else:
    print(f"🔄 Starting preprocessing: {raw_files} files")
    print(f"(This will take ~10-20 minutes depending on Colab speed)")
    
    # Preprocess all files
    for filename in tqdm(sorted(os.listdir(DRIVE_RAW_DATA))):
        if not filename.endswith('.npy'):
            continue
        
        input_path = os.path.join(DRIVE_RAW_DATA, filename)
        output_path = os.path.join(OUTPUT_PREPROCESSED, filename)
        
        # Skip if already processed
        if os.path.exists(output_path):
            continue
        
        try:
            # Load raw landmarks
            landmarks = np.load(input_path)  # (T, 138)
            
            # Preprocess
            preprocessed = preprocess_single_file(landmarks)  # (T, 414)
            
            # Save
            np.save(output_path, preprocessed)
        except Exception as e:
            print(f"⚠️ Error processing {filename}: {e}")
    
    print(f"✅ Preprocessing complete! {len(os.listdir(OUTPUT_PREPROCESSED))} files saved")

# Copy CSV to Colab if not there
if not os.path.exists('/content/iSign_v1.1.csv'):
    shutil.copy(CSV_PATH, '/content/iSign_v1.1.csv')
    print("✅ CSV copied to /content/")


In [ ]:
# ========== CONVERT ALL PREPROCESSED DATA TO SINGLE HDF5 FILE ==========
import h5py
import shutil
from pathlib import Path

H5_OUTPUT_PATH = "/content/dataset.h5"

# Check if HDF5 already created
if os.path.exists(H5_OUTPUT_PATH):
    print(f"✅ HDF5 file already exists at {H5_OUTPUT_PATH}")
    with h5py.File(H5_OUTPUT_PATH, 'r') as f:
        print(f"   Samples in H5: {len(f['landmarks'])}")
        print(f"   Feature dim: {f['landmarks'].shape[1]}")
else:
    print(f"🔄 Converting preprocessed .npy files to single HDF5 file...")
    print(f"   Output: {H5_OUTPUT_PATH}")
    
    # First, ensure all files are preprocessed
    preprocessed_files = sorted([f for f in os.listdir(OUTPUT_PREPROCESSED) if f.endswith('.npy')])
    print(f"   Found {len(preprocessed_files)} preprocessed files")
    
    # Create HDF5 file with chunked storage for efficiency
    with h5py.File(H5_OUTPUT_PATH, 'w') as h5file:
        # Pre-calculate total size for efficient allocation
        total_frames = 0
        file_info = []  # Store (filename, start_idx, num_frames)
        
        # First pass: scan all files to get sizes
        print("   Scanning file sizes...")
        for filename in tqdm(preprocessed_files):
            filepath = os.path.join(OUTPUT_PREPROCESSED, filename)
            try:
                data = np.load(filepath)
                num_frames = data.shape[0]
                file_info.append((filename, total_frames, num_frames))
                total_frames += num_frames
            except Exception as e:
                print(f"   ⚠️ Error reading {filename}: {e}")
        
        print(f"   Total frames: {total_frames}")
        
        # Create datasets with proper chunking
        feature_dim = 414  # Your feature dimension
        
        # Main data storage
        landmarks_dataset = h5file.create_dataset(
            'landmarks',
            shape=(total_frames, feature_dim),
            dtype='float32',
            chunks=(256, feature_dim),  # Chunk size: 256 frames at a time
            compression='gzip',
            compression_opts=4
        )
        
        # Store file mapping for later retrieval
        filenames_dataset = h5file.create_dataset(
            'filenames',
            (len(file_info),),
            dtype=h5py.string_dtype(encoding='utf-8')
        )
        
        # Store frame indices for each file (start, count)
        frame_indices = h5file.create_dataset(
            'frame_indices',
            shape=(len(file_info), 2),
            dtype='int32'
        )
        
        # Second pass: load and store data
        print("   Loading and storing data...")
        for idx, (filename, start_idx, num_frames) in enumerate(tqdm(file_info)):
            filepath = os.path.join(OUTPUT_PREPROCESSED, filename)
            data = np.load(filepath)
            
            # Store the data
            landmarks_dataset[start_idx:start_idx + num_frames] = data
            
            # Store metadata
            filenames_dataset[idx] = filename
            frame_indices[idx] = [start_idx, num_frames]
        
        # Create metadata group
        meta = h5file.create_group('metadata')
        meta.attrs['total_samples'] = len(file_info)
        meta.attrs['total_frames'] = total_frames
        meta.attrs['feature_dim'] = feature_dim
        meta.attrs['normalization'] = 'applied'
        meta.attrs['has_velocity'] = True
        meta.attrs['has_acceleration'] = True
        
        print(f"✅ HDF5 file created successfully!")
        print(f"   File size: {os.path.getsize(H5_OUTPUT_PATH) / 1e9:.2f} GB")
        print(f"   Total samples: {len(file_info)}")
        print(f"   Total frames: {total_frames}")


In [ ]:
# Import model and training components from your repo
from model_hybrid import HybridCTCAttentionModel

# Test model creation
model = HybridCTCAttentionModel(
    input_dim=414,
    hidden_dim=384,
    vocab_size=73
).cuda()

print(f"Model parameters: {model.get_num_params():,}")
print(f"Model size: {model.get_model_size_mb():.2f} MB")

# Quick forward test
with torch.no_grad():
    x = torch.randn(4, 100, 414).cuda()
    lens = torch.tensor([100, 90, 80, 70]).cuda()
    tgt = torch.randint(0, 73, (4, 20)).cuda()
    ctc_out, attn_out = model(x, lens, tgt)
    print(f"CTC output: {ctc_out.shape}")
    print(f"Attn output: {attn_out.shape}")
print("✅ Model works!")

In [ ]:
# ========== HDF5 DATASET CLASS FOR EFFICIENT LOADING ==========
import h5py
from torch.utils.data import Dataset
import pandas as pd

class HDF5SignLanguageDataset(Dataset):
    """Dataset that loads landmarks from a single HDF5 file"""
    
    def __init__(self, h5_path, csv_path, vocab_path, max_src_len=500, max_tgt_len=100, 
                 indices=None, is_train=True):
        """
        Args:
            h5_path: Path to the HDF5 file
            csv_path: Path to the CSV with sign labels
            vocab_path: Path to vocabulary pickle file
            max_src_len: Max source (landmark) sequence length
            max_tgt_len: Max target (text) sequence length
            indices: List of file indices to use (for train/val split)
            is_train: Whether this is training set
        """
        self.h5_path = h5_path
        self.max_src_len = max_src_len
        self.max_tgt_len = max_tgt_len
        self.is_train = is_train
        
        # Load vocabulary
        import pickle
        with open(vocab_path, 'rb') as f:
            self.vocab = pickle.load(f)
        
        # Load CSV
        self.df = pd.read_csv(csv_path)
        
        # Get file mapping from HDF5
        with h5py.File(h5_path, 'r') as f:
            self.filenames = [f['filenames'][i].decode('utf-8') for i in range(len(f['filenames']))]
            self.frame_indices = f['frame_indices'][:]  # (N_files, 2) with [start, count]
        
        # Use provided indices or all files
        if indices is None:
            self.indices = list(range(len(self.filenames)))
        else:
            self.indices = indices
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        file_idx = self.indices[idx]
        filename = self.filenames[file_idx]
        
        # Get frame range for this file
        start_frame, num_frames = self.frame_indices[file_idx]
        
        # Load landmarks from HDF5
        with h5py.File(self.h5_path, 'r') as f:
            # Load all frames for this file
            landmarks = f['landmarks'][start_frame:start_frame + num_frames]  # (T, 414)
        
        # Crop or pad to max_src_len
        if landmarks.shape[0] > self.max_src_len:
            landmarks = landmarks[:self.max_src_len]
        else:
            pad_len = self.max_src_len - landmarks.shape[0]
            landmarks = np.pad(landmarks, ((0, pad_len), (0, 0)), mode='constant')
        
        # Get label from CSV (match filename to video_id)
        video_id = filename.replace('.npy', '')
        try:
            row = self.df[self.df['video_id'] == video_id].iloc[0]
            label = row['text']  # or 'gloss' depending on your CSV
        except:
            label = ""
        
        # Convert label to token indices
        tokens = self.vocab.get(label, [])  # Returns list of token indices
        tokens = tokens[:self.max_tgt_len]
        
        # Pad tokens
        token_len = len(tokens)
        tokens = np.array(tokens + [0] * (self.max_tgt_len - len(tokens)), dtype=np.int64)
        
        return {
            'landmarks': torch.FloatTensor(landmarks),
            'landmarks_len': torch.LongTensor([landmarks.shape[0]]),
            'tokens': torch.LongTensor(tokens),
            'tokens_len': torch.LongTensor([token_len]),
            'video_id': video_id
        }

print("✅ HDF5Dataset class loaded!")


In [ ]:
# Import training components
from train_hybrid import (
    SignLanguageDataset, 
    collate_fn, 
    HybridLoss,
    Trainer
)

# Patch the Trainer to use Colab config
class ColabTrainer(Trainer):
    def __init__(self):
        # Use Colab config instead
        super().__init__(ColabConfig())

print("Training components imported!")

In [ ]:
# Create trainer and start training!
trainer = ColabTrainer()

print(f"Train samples: {len(trainer.train_dataset)}")
print(f"Val samples: {len(trainer.val_dataset)}")
print(f"Batches per epoch: {len(trainer.train_loader)}")

In [ ]:
# Start training!
# To resume from checkpoint: trainer.train(resume_from="/content/drive/MyDrive/kortex_checkpoints/latest.pth")
trainer.train()

In [ ]:
# Visualize training (run after training or during breaks)
import matplotlib.pyplot as plt

history = trainer.history

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Loss
axes[0,0].plot(history['train_loss'], label='Train')
axes[0,0].plot(history['val_loss'], label='Val')
axes[0,0].set_title('Total Loss')
axes[0,0].legend()

# Accuracy
axes[0,1].plot([a*100 for a in history['train_acc']], label='Train')
axes[0,1].plot([a*100 for a in history['val_acc']], label='Val')
axes[0,1].set_title('Accuracy (%)')
axes[0,1].legend()

# CTC Loss
axes[1,0].plot(history['train_ctc_loss'], label='Train CTC')
axes[1,0].plot(history['val_ctc_loss'], label='Val CTC')
axes[1,0].set_title('CTC Loss')
axes[1,0].legend()

# Attention Loss
axes[1,1].plot(history['train_attn_loss'], label='Train Attn')
axes[1,1].plot(history['val_attn_loss'], label='Val Attn')
axes[1,1].set_title('Attention Loss')
axes[1,1].legend()

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/kortex_checkpoints/training_curves.png')
plt.show()

print(f"Best Val Loss: {min(history['val_loss']):.4f}")
print(f"Best Val Acc: {max(history['val_acc'])*100:.2f}%")

In [ ]:
# Save final model to Drive
!cp /content/kortex_5th_sem/sign_to_text/checkpoints_hybrid/* /content/drive/MyDrive/kortex_checkpoints/
print("Checkpoints saved to Google Drive!")